In [26]:
import os
from dotenv import load_dotenv

引入 OPENAI_API_KEY 和 OPENAI_API_BASE 接口代理地址

In [24]:
load_dotenv()
OPENAI_API_KEY = os.environ['OPENAI_API_KEY']
OPENAI_API_BASE = os.environ['OPENAI_API_BASE']

使用 PyPDFLoader 解析 PDF，需要先安装 pypdf 库

In [27]:
from langchain.document_loaders import PyPDFLoader

loader = PyPDFLoader('pdf_data/893.pdf')
pages = loader.load_and_split()

把解析好的文本内容用 text_splitter 分块

In [28]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
# chunk_overlap=0 表示没有重叠的字符
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000, chunk_overlap=0)
split_docs = text_splitter.split_documents(pages)
print(f'{len(split_docs)} split_docs')

35 split_docs


In [29]:
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import Chroma

embeddings = OpenAIEmbeddings(openai_api_key=OPENAI_API_KEY)

persist_directory = "chroma_storage_893"

if os.path.exists(persist_directory) != True:
    # 把分块内容处理成 embeddings
    vectorstore = Chroma.from_documents(
        split_docs, embeddings, persist_directory=persist_directory)
    # 结果持久化
    vectorstore.persist()

In [30]:
query = 'ChatGPT 是如何发展演变的，历史沿革是怎样的？'
# 加载保存在本地的文档数据
vectordb = Chroma(persist_directory=persist_directory,
                  embedding_function=embeddings)
# 查询与提问最相关的2个文档
search_docs = vectordb.similarity_search(query, 2)
print(search_docs)

Using embedded DuckDB with persistence: data will be stored in: chroma_storage_893


[Document(page_content='浅析 ChatGPT : 历史沿革 、应用现状及前景展望\n刘禹良1，李鸿亮2，白翔1*，金连文2\n1. 华中科技大学人工智能与自动化学院 ，武汉  430074 ； 2. 华南理工大学电子与信息学院 ，广州  510640\n摘 要： 近年来 ，人工智能技术接连取得突破 ，尤其是在强化学习 、大规模语言模型和人工智能生成内容技术等方\n面，正逐步成为各个行业的创新驱动力 。OpenAI 于2022 年11月30日发布的 ChatGPT 由于具有惊人的自然语言理\n解和生成能力 ，引起全社会大范围的关注 ，成为全球热议的话题 ，并被广泛应用于各个行业 。仅两个月后 ，ChatGPT\n的月活跃用户数便达 1亿，成为史上用户数增长最快的消费者应用 。鉴于 ChatGPT 目前造成的影响 ，对其进行全面\n的分析较为必要 。本文从历史沿革 、应用现状和前景展望这 3个角度对 ChatGPT 进行剖析 ，探究其对社会的影响 、技\n术的原理和挑战以及未来发展的可能性 ，并从模型能力的角度简要介绍 GPT-4相对于 ChatGPT 的改进 。作为一个\n现象级技术产品 ，从技术角度而言 ChatGPT 对相关领域具有里程碑式的重要意义 ，从应用角度而言其可能会给人类\n社会带来巨大的影响 。ChatGPT 有潜力成为计算机领域最伟大的成就之一 。但就目前而言 ，ChatGPT 仍然存在一些\n局限 ，尚未达到强人工智能的水平 。在当前阶段 ，研究人员需要对人工智能技术持有自信和谦虚学习的态度 ，继续\n发展相关的技术研究和应用 。\n关键词： 人工智能 （AI） ；深度学习 ；自然语言处理 ；生成式人工智能技术 （AIGC ） ；ChatGPT\nA brief analysis of ChatGPT ： historical evolution ， \ncurrent applications ， and future prospects\nLiu\u3000Yuliang1， Li\u3000Hongliang2， Bai\u3000Xiang1*， Jin\u3000Lianwen2\n1. School of Artificial Intelligence and Automation ， Huazhong Univ

In [31]:
from langchain.chat_models import ChatOpenAI
from langchain.chains.question_answering import load_qa_chain
# 构建llm 
# temperature=0 意味着需要最保守的回答 减小模型产生幻觉的几率
llm = ChatOpenAI(temperature=0, openai_api_key=OPENAI_API_KEY,
                 model_name='gpt-3.5-turbo', api_base=OPENAI_API_BASE)
chain = load_qa_chain(llm, chain_type='stuff')
results = chain.run(input_documents=search_docs, question=query)
print(f'Q: {query}')
print(f'A: {results}')

Q: ChatGPT 是如何发展演变的，历史沿革是怎样的？
A: 根据文中所述，ChatGPT是一种基于生成式预训练技术的语言模型，它使用了Transformer架构和GPT（生成式预训练技术）来预测下一个单词的概率分布，以生成自然语言文本。GPT训练的模型是一种应用于自然语言处理（NLP）的模型，它通过在超大型文本语料库上训练学习到的语言模式来实现的。从2018年拥有1.17亿参数的GPT-1到2020年拥有1750亿参数的GPT-3，OpenAI的语言模型智能化程度明显提升。在2022年1月基于RLHF（reinforcement learning from human feedback）的InstructGPT的提出显著降低了有害、不真实和有偏差输出的概率。在2022年12月，ChatGPT作为InstructGPT的姐妹模型被推出。因此，ChatGPT是在GPT-3的基础上发展演变而来的。


In [32]:
import xml.etree.ElementTree as ET

speak = ET.Element('speak')
speak.set('xmlns', 'http://www.w3.org/2001/10/synthesis')
speak.set('xmlns:mstts', 'http://www.w3.org/2001/mstts')
speak.set('xmlns:emo', 'http://www.w3.org/2009/10/emotionml')
speak.set('version', '1.0')
speak.set('xml:lang', 'en-US')

voice = ET.SubElement(speak, 'voice')
voice.set('name', 'zh-TW-YunJheNeural')

prosody = ET.SubElement(voice, 'prosody')
prosody.set('rate', '20%')
prosody.set('pitch', '20%')
prosody.text = query + results

ET.dump(speak)
tree = ET.ElementTree(speak)
tree.write(persist_directory + '/SSML.xml', encoding='utf-8')

<speak xmlns="http://www.w3.org/2001/10/synthesis" xmlns:mstts="http://www.w3.org/2001/mstts" xmlns:emo="http://www.w3.org/2009/10/emotionml" version="1.0" xml:lang="en-US"><voice name="zh-TW-YunJheNeural"><prosody rate="20%" pitch="20%">ChatGPT 是如何发展演变的，历史沿革是怎样的？根据文中所述，ChatGPT是一种基于生成式预训练技术的语言模型，它使用了Transformer架构和GPT（生成式预训练技术）来预测下一个单词的概率分布，以生成自然语言文本。GPT训练的模型是一种应用于自然语言处理（NLP）的模型，它通过在超大型文本语料库上训练学习到的语言模式来实现的。从2018年拥有1.17亿参数的GPT-1到2020年拥有1750亿参数的GPT-3，OpenAI的语言模型智能化程度明显提升。在2022年1月基于RLHF（reinforcement learning from human feedback）的InstructGPT的提出显著降低了有害、不真实和有偏差输出的概率。在2022年12月，ChatGPT作为InstructGPT的姐妹模型被推出。因此，ChatGPT是在GPT-3的基础上发展演变而来的。</prosody></voice></speak>
